In [ ]:
import os
import subprocess

def git_repo_root():
    # Run the 'git rev-parse --show-toplevel' command to get the root directory of the Git repository
    try:
        root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel'], universal_newlines=True).strip()
        return root
    except subprocess.CalledProcessError:
        # Handle the case where the current directory is not inside a Git repository
        return None

# Get the root directory of the Git repository
git_root = git_repo_root()

if git_root:
    # Change the working directory to the root of the Git repository
    os.chdir(git_root)
    print(f"Changed working directory to: {git_root}")
else:
    print("Not inside a Git repository.")

In [ ]:
from diffusion import VPSDE
import torchvision
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from diffusion import VPSDE
from torch.utils.data import DataLoader
from torch.optim import Adam
from guided_diffusion import  train_classification_network_mnist, Net

In [ ]:
device = 'cuda'
data = 'MNIST'
model = Net().to(device)
sde = VPSDE(100, 0.1, 20, device = device)

''' hyper params'''
n_epochs =   50
batch_size =  32
lr=1e-4
optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
tfm = transforms.Compose([transforms.ToTensor(), transforms.Pad(2)])
data = torchvision.datasets.MNIST(f'./data/', transform=tfm, download = True)
data_loader = DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
train_classification_network_mnist(data_loader, model, sde, optimizer, epochs=100)

In [ ]:
import torch

def prediction_accurary(time):
    accs = []

    num = 5
    for x, y in data_loader:
      x=x.to('cuda')
      y=y.to('cuda')
      random_t = torch.ones(x.shape[0], device='cuda') * time
      z = torch.randn_like(x, device = 'cuda')
      mu, std= sde.marginal(x, random_t)
      perturbed_x=mu+std*z

      #print(random_t.device, perturbed_x.device)

      pred = model(perturbed_x, random_t)

      acc = (y==torch.argmax(pred, dim = 1)).sum() / x.shape[0]
      accs+=[acc]

      if num < 25:
        break
      num-=1

    return sum(accs) / len(accs)


prediction_accurary(0.1)